In [1]:
#!pip install torch
#!pip install torchtext
#!pip install openpyxl
#!pip install nltk
#!pip install spacy
!python -m spacy download en_core_web_md 

2023-08-17 07:36:07.340117: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 07:36:07.800011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-17 07:36:08.314172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-17 07:36:08.314431: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /data/home/ayyoobmohd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /data/home/ayyoobmohd/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import numpy as np
import pandas as pd
import random
import torch
import os
import spacy
from torchtext.vocab import GloVe, FastText
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import nltk
import matplotlib.pyplot as plt

2023-08-17 07:36:18.870431: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 07:36:19.408709: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-17 07:36:19.867679: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-17 07:36:19.867996: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [4]:
data = pd.read_excel('/data/home/ayyoobmohd/DLNLP/Glove-and-Sentiments/Datasets/ClassificationDataset1.xlsx')
#/data/home/ayyoobmohd/DLNLP/Glove-and-Sentiments/Datasets

In [5]:
print(data.head(5))
print(data.shape)
#df = df.head(500)

                                     Negative Review  \
0  I am so angry that i made this post available ...   
1                                        No Negative   
2   Rooms are nice but for elderly a bit difficul...   
3   My room was dirty and I was afraid to walk ba...   
4  You When I booked with your company on line yo...   

                                     Positive Review  
0   Only the park outside of the hotel was beauti...  
1   No real complaints the hotel was great great ...  
2   Location was good and staff were ok It is cut...  
3   Great location in nice surroundings the bar a...  
4    Amazing location and building Romantic setting   
(8999, 2)


## Download Glove vectors and preprocess data

In [6]:
global_vectors = GloVe(name='840B', dim=300)
fasttext = FastText(language="en")

In [7]:
## Declaring Path 
#/data/home/ayyoobmohd/DLNLP/Glove-and-Sentiments/Datasets/ClassificationDataset1.xlsx
pos_path = '/data/home/ayyoobmohd/DLNLP/Glove-and-Sentiments/data/positive_reviews.csv'
neg_path = '/data/home/ayyoobmohd/DLNLP/Glove-and-Sentiments/data/negative_reviews.csv'

def making_new_dataset(data):
    data.to_csv(pos_path, index = False, header= False,
          encoding = "latin-1", columns = ['Positive Review'])
    
    data.to_csv(neg_path, index = False, header= False,
          encoding = "latin-1", columns = ['Negative Review'])
    
    
    positive_set = open(pos_path, "r", encoding="latin-1").read()
    negative_set = open(neg_path, "r", encoding="latin-1").read()
    
    pos_set = positive_set.split("\n")[:-1]
    neg_set = negative_set.split("\n")[:-1]
    
    #print(len(positive_data), len(negative_data))
    
    return pos_set, neg_set

positive_data, negative_data = making_new_dataset(data) 

In [8]:
def preprocess_ReviewsText(data, mode='positive'):
    
    nlp = spacy.load("en_core_web_md")
    data_set = []
    vocabulary = {}
    chars_to_remove = ['--', '`', '~', '<', '>', '*', '{', '}', '^', '=', '_', '[', ']', '|', '- ', '.', ',']
    
    if mode == 'positive':
        sentiment = 1
    else :
        sentiment = 0
    
    for i, v in enumerate(data):

            # Removing Un-necessary symbols in our sentence
            for chars in chars_to_remove:
                v = v.replace(chars, " ", -1)

            sentence = torch.zeros(600)
            n = 0
            for token in nlp(v): 

                sentence[:300] += global_vectors.get_vecs_by_tokens(token.text, lower_case_backup=True)
                sentence[300:] += fasttext.get_vecs_by_tokens(token.text, lower_case_backup=True).squeeze(dim=0)
                n += 1

            # Taking mean
            sentence = sentence / n
            data_set.append((sentence, sentiment))
    
    return data_set     


In [9]:
pos = preprocess_ReviewsText(positive_data, 'positive')
neg = preprocess_ReviewsText(negative_data, 'negative')
data = pos + neg
print(f"Length of positive reviews : {len(pos)} || Length of positive reviews : {len(neg)}")
print(f"Length of Complete concatenated Dataset of reviews : {len(data)}")

Length of positive reviews : 8999 || Length of positive reviews : 8999
Length of Complete concatenated Dataset of reviews : 17998


In [10]:
with open('temp.txt', 'w') as fp:
    fp.write('\n'.join('%s %s' % x for x in data))

## Utility Functions

In [11]:
def set_seed(seed = 42):
    '''
        For Reproducibility: Sets the seed of the entire notebook.
    '''

    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    # Sets a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

def split_indices(num_values, percentage):

    # Determine size of Validation set
    val_size = int(percentage * num_values)

    # Create random permutation of 0 to num_values-1
    idxs = np.random.permutation(num_values)
    return idxs[val_size:], idxs[:val_size]

set_seed(1)
train_pos_indices, val_pos_indices = split_indices(len(positive_data), 0.1)
train_neg_indices, val_neg_indices = split_indices(len(negative_data), 0.1)

train_indices = np.concatenate((train_pos_indices, train_neg_indices+len(positive_data)-1))
val_indices = np.concatenate((val_pos_indices, val_neg_indices+len(positive_data)-1))

In [12]:
len(val_indices), len(train_indices)

(1798, 16200)

## Data Batch

In [13]:
def batch_fn(instn):

    sentence = torch.zeros(len(instn), 600)

    for i, v in enumerate(instn):
        sentence[i] = v[0]

    labels = torch.Tensor([x[1] for x in instn])

    return (sentence, labels)

batch_size = 64

train_sampler   = SubsetRandomSampler(train_indices)
train_loader    = DataLoader(data, batch_size, sampler=train_sampler, collate_fn=batch_fn)

val_sampler     = SubsetRandomSampler(val_indices)
val_loader      = DataLoader(data, batch_size, sampler=val_sampler, collate_fn=batch_fn)

# Models : DAN


In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
class DAN(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Linear(600, 1024)
        self.linear2 = nn.Linear(1024, 2048)
        self.linear3 = nn.Linear(2048, 512)
        self.linear4 = nn.Linear(512, 64)
        self.linear5 = nn.Linear(64, 1)

    def forward(self, Xb, tsne = False):
        x = self.linear1(Xb)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x) 
        x = self.linear3(x)
        x = F.relu(x)
        x = self.linear4(x)
        
        if tsne == True:
            return x
            
        x = F.relu(x)
        x = self.linear5(x)
        x = torch.sigmoid(x)
        return x

In [16]:
model = DAN()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
loss_fn = F.binary_cross_entropy

In [17]:
# !pip install torchvision

In [18]:
from helper_functions import plot_predictions, plot_decision_boundary, accuracy_fn

In [19]:
max_epoch = 25

best_test_acc = 0
for ep in range(max_epoch):

    epoch_loss = 0

    model.train()

    for xb, yb in tqdm(train_loader):
        xb = xb.to(device)
        yb = yb.to(device)

        y_hat = model(xb)

        loss = loss_fn(y_hat.squeeze(), yb)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss += loss

    print("Epoch: ", ep+1, " Training Loss: ", epoch_loss.item())
    
    #----------- Validation -----------

    val_labels = []
    val_pred = []

    model.eval()

    val_epoch_loss = 0

    for xb, yb in tqdm(val_loader):
        xb = xb.to(device)
        yb = yb.to(device)

        y_hat = model(xb)

        loss = loss_fn(y_hat.squeeze(), yb)

        val_epoch_loss += loss

        val_labels.extend(yb.cpu().detach().numpy())
        val_pred.extend(y_hat.round().cpu().detach().numpy())

    print("Validation loss: ", val_epoch_loss.item())
    print("Validation accuracy: ", accuracy_score(val_labels, val_pred))


    if ep > 15 and prev_val_loss - val_epoch_loss.item() > 0.05:
        print("Saving Model")
        torch.save(model.state_dict(), "DAN_model1.pt")
    
    prev_val_loss = val_epoch_loss.item()


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 254/254 [00:00<00:00, 440.10it/s]


Epoch:  1  Training Loss:  69.30224609375


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 1581.50it/s]


Validation loss:  5.182392597198486
Validation accuracy:  0.9327030033370411


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 254/254 [00:00<00:00, 794.55it/s]


...

Validation loss:  8.328694343566895
Validation accuracy:  0.9399332591768632


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 254/254 [00:00<00:00, 477.06it/s]


Epoch:  25  Training Loss:  9.107155799865723


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 1750.54it/s]


Validation loss:  7.744479656219482
Validation accuracy:  0.9371523915461624
Saving Model


## Model : LSTM

In [27]:
from torchtext.data import get_tokenizer
global_vectors = GloVe(name='840B', dim=300)

def preprocess_tokenized_reviewText(data, mode='positive'):
    
    nlp = spacy.load("en_core_web_md")
    data_set = []
    vocab = []
    chars_to_remove = ['--', '`', '~', '<', '>', '*', '{', '}', '^', '=', '_', '[', ']', '|', '- ', '.', ',']
    tokenizer = get_tokenizer("basic_english")
    
    if mode == 'positive':
        sentiment = 1
    else :
        sentiment = 0
    
    for line in data:
        # Tokenizes the input text into words
        tokens = tokenizer(line)

        data_set.append((tokens, sentiment))
        # Adds the extracted words to a list
        vocab.extend(tokens)
    print(f"--- {mode} Finished ---")
    
    return data_set

#-----------------------------------------------------------------------------------#

#len(set(vocab))
def sort_key(s):
    return len(s[0])

In [26]:
new_pos = preprocess_tokenized_reviewText(positive_data, 'positive')
new_neg = preprocess_tokenized_reviewText(negative_data, 'negative')
new_data = new_pos + new_neg
print(f"Length of positive reviews : {len(new_pos)} || Length of positive reviews : {len(new_neg)}")
print(f"Length of Complete concatenated Dataset of reviews : {len(new_data)}")

--- positive Finished ---
--- negative Finished ---
Length of positive reviews : 8999 || Length of positive reviews : 8999
Length of Complete concatenated Dataset of reviews : 17998
